# Fitting a Carbonate Correction for MP-sourced Gibbs free energy of formation data (300 K)

Author: Matthew McDermott

Date: May 24, 2022

#### Imports

In [20]:
from mp_api import MPRester
from pymatgen.core.composition import Composition, Element

import numpy as np
import pandas
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import minimize

from rxn_network.entries.entry_set import GibbsEntrySet

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Chemical systems

In [2]:
metals = ["Ag", "Ba", "Ca", "Cd", "Cs", "Fe", "K", 
          "Li", "Mg", "Mn", "Na", "Pb", "Rb", "Sr", "Zn"]  # all metals with carbonate energies available

systems = [(m, "C", "O") for m in metals]

### Downloading Materials Project (MP) data using API

In [3]:
all_entries = {}

for system in systems:
    with MPRester() as mpr:
        entries = mpr.get_entries_in_chemsys(system)
        
    all_entries[system[0]] = entries

/Users/mcdermott/opt/miniconda3/envs/rn_dev/lib/python3.9/site-packages/mp_api/client.py:138: builtins.UserWarning: Problem loading MPContribs client: Duplicate operationId: download_entries


Retrieving ThermoDoc documents:   0%|          | 0/136 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/169 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/170 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/121 [00:00<?, ?it/s]

...

Retrieving ThermoDoc documents:   0%|          | 0/174 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/144 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/170 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/147 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/132 [00:00<?, ?it/s]

### Acquiring GibbsComputedEntries and corresponding experimental entries

In [120]:
mp_entries = []
exp_entries = []

temp = 300  # Kelvin

for m, entries in all_entries.items():
    gibbs = GibbsEntrySet.from_entries(entries, temp, include_nist_data=False, apply_carbonate_correction=False)
    gibbs_exp = GibbsEntrySet.from_entries(entries, temp, include_nist_data=True, include_freed_data=True, apply_carbonate_correction=False)
    
    oxidation_states = Element(m).common_oxidation_states
    
    for os in oxidation_states:
        formula = Composition(f"{m}2(CO3){os}").reduced_formula

        mp_entry = None
        exp_entry = None
        
        try:
            mp_entry = gibbs.get_min_entry_by_formula(formula)
            exp_entry = gibbs_exp.get_min_entry_by_formula(formula)
        except:
            continue
        
        if exp_entry.is_experimental:
            mp_entries.append(mp_entry)
            exp_entries.append(exp_entry)

In [121]:
energies_calc = []
energies_exp = []
names = []

for e_exp, e_calc in zip(exp_entries, mp_entries):
    formula = e_exp.composition.reduced_formula
    num_atoms_per_fu = Composition(formula).num_atoms
    
    energy_exp = e_exp.energy_per_atom * num_atoms_per_fu
    energy_calc = e_calc.energy_per_atom * num_atoms_per_fu
    name = e_calc.composition.reduced_formula
    
    energies_exp.append(energy_exp)
    energies_calc.append(energy_calc)
    names.append(name)

### Plot before correction

In [128]:
df = pandas.DataFrame({"x":energies_exp, "y":energies_calc, "name":names})

scatter = px.scatter(df, x="x", y="y", hover_name="name", symbol="name", color="name")
line = px.line(x=[-12, -4], y=[-12, -4])

fig = go.Figure(scatter.data + line.data)
fig.update_layout({"title": "Carbonate formation energies",
                   "xaxis_title":"Experimental dGf (eV/f.u.)",
                   "yaxis_title":"Calculated dGf (eV/f.u.)"})
fig.show()

### Fitting correction with scipy.optimize.minimize
Minimizing the mean absolute error (MAE) for all corrected phases.

In [124]:
def func(correction):
    return np.mean(np.abs((df["y"] + correction) - df["x"]))  # definition of MAE

In [125]:
final_correction = minimize(func, 0.25).x[0]

In [126]:
df_new = df.copy()
df_new["y"] = df_new["y"] + final_correction

scatter_new = px.scatter(df_new, x="x", y="y", hover_name="name", symbol="name", color="name")
line_new = px.line(x=[-12, -4], y=[-12, -4])

fig_new = go.Figure(scatter_new.data + line_new.data)
fig_new.update_layout({"title": "Corrected carbonate formation energies",
                   "xaxis_title":"Experimental dGf (eV/f.u.)",
                   "yaxis_title":"Calculated dGf (eV/f.u.)"})
fig_new.show()

## Final correction value (eV per CO$_3^{2-}$):

In [127]:
print(final_correction)

0.8281951505892449


### Test carbonate correction class

In [105]:
li_entries = GibbsEntrySet.from_entries(all_entries["Li"], 300, include_nist_data=False)

In [111]:
li2co3 = li_entries.get_min_entry_by_formula("Li2CO3")
li2co3.energy_adjustments

[ConstantEnergyAdjustment:
   Name: Gibbs SISSO Correction
   Value: 1.868 eV
   Uncertainty: 0.600 eV
   Description: Gibbs correction: dGf(300 K) - dHf (298 K) (1.868 eV)
   Generated by: None,
 CarbonateCorrection:
   Name: Carbonate Correction
   Value: 1.656 eV
   Uncertainty: nan eV
   Description: Correction for dGf with (CO3)2- anion, as fit to MP data (300 K). (0.828 eV/atom x 2.0 atoms)
   Generated by: None]

In [135]:
mp_entries = []
exp_entries = []

temp = 300  # Kelvin

for m, entries in all_entries.items():
    gibbs = GibbsEntrySet.from_entries(entries, temp, include_nist_data=False, apply_carbonate_correction=True)
    gibbs_exp = GibbsEntrySet.from_entries(entries, temp, include_nist_data=True, include_freed_data=True, apply_carbonate_correction=False)
    
    oxidation_states = Element(m).common_oxidation_states
    
    for os in oxidation_states:
        formula = Composition(f"{m}2(CO3){os}").reduced_formula

        mp_entry = None
        exp_entry = None
        
        try:
            mp_entry = gibbs.get_min_entry_by_formula(formula)
            exp_entry = gibbs_exp.get_min_entry_by_formula(formula)
        except:
            continue
        
        if exp_entry.is_experimental:
            mp_entries.append(mp_entry)
            exp_entries.append(exp_entry)

In [136]:
energies_calc = []
energies_exp = []
names = []

for e_exp, e_calc in zip(exp_entries, mp_entries):
    energy_exp = e_exp.energy_per_atom
    energy_calc = e_calc.energy_per_atom
    name = e_calc.composition.reduced_formula
    
    energies_exp.append(energy_exp)
    energies_calc.append(energy_calc)
    names.append(name)
    
df = pandas.DataFrame({"x":energies_exp, "y":energies_calc, "name":names})

scatter = px.scatter(df, x="x", y="y", hover_name="name", symbol="name", color="name")
line = px.line(x=[-2.75, -0.75], y=[-2.75, -0.75])

fig = go.Figure(scatter.data + line.data)
fig.update_layout({"title": "Test of Carbonate Correction",
                   "xaxis_title":"Experimental dGf (eV/atom)",
                   "yaxis_title":"Calculated dGf (eV/atom)"})
fig.show()